In [3]:
import pandas as pd
import sys
sys.path.append('python/')

In [15]:
from glob import glob
from datedFiles import *
from localDataAccess import *

files_original = glob('python/tests/unit/fixtures/original/*')
files_wordindex = glob('python/tests/unit/fixtures/wordindex/*')
localdata = LocalDataAccess('python/tests/unit/fixtures/original')
data1 =  pd.read_json(files_original[0], orient='records', lines=True)
dateddatafiles = DatedDataFiles(files_original, id_varname = 'id', date_prefix='records_')
base_timestamp = pd.Timestamp('2020-08-25')
datedfilenamefilter = DatedFilenameFilter(base_timestamp, days=2, no_newer=True)
datedwordindexfilter = DatedWordindexFilter(files_wordindex, 'protest')
# dateddatafiles.apply_id_filter(datedwordindexfilter)
# dataloaderjson = DataLoaderJson(orient='records', lines=True)
# dataloaderjson.load_data(dateddatafiles, localdata)
# dataloadercsv = DataLoaderCSV()


In [9]:
from factory import Factory
#from datedFiles import *
from dataLoaderMethods import *

data_loader_factory = Factory()
data_loader_factory.register_tool('json', DataLoaderJson)
data_loader_factory.register_tool('csv', DataLoaderCSV)


class DataLoader:

    def __init__(self, DataAccess):
        self.DataAccess = DataAccess

    def load(self, DataFiles, FileFilter=None, IndexFilter=None):
        if not FileFilter: DataFiles.apply_file_filter(FileFilter)
        if not IndexFilter: DataFiles.apply_id_filter(IndexFilter)
        data_loader = data_loader_factory.get_tool(DataFiles.file_format)
        data_loader.load_data(DataFiles, self.DataAccess)
        return data_loader.df



In [11]:
dataloader = DataLoader(localdata)

In [16]:
result = dataloader.load(dateddatafiles, datedfilenamefilter, datedwordindexfilter)


In [18]:
len(result)

283112

In [19]:
class placeholder():
    pass

In [20]:
a = placeholder

In [21]:
a.a, a.b, a.c = (1,2,3)

In [23]:
(a.a, a.b, a.c)


(1, 2, 3)

In [122]:
def calc_stat_sentiments(df_sentiments):
    stat = (df_sentiments[['created_at_h', 'compound']]
     .groupby('created_at_h')
     .agg(['mean','count'])
     .round(3)
     .sort_values("created_at_h") 
     .reset_index()
    )

    stat.columns =  stat.columns.droplevel()
    stat = stat.rename(columns ={'': 'time'})
    return stat

def convert_vars_to_numeric(df, vars):
    for var in list(vars):
        df[var] = pd.to_numeric(df[var])
    return None


def calc_stat_emotions(df_emotions):
    emo_labels = [*df_emotions.columns][2:]
    convert_vars_to_numeric(df_emotions, emo_labels)

    stat = (df_emotions[['created_at_h', *emo_labels]]
     .groupby('created_at_h')
     .agg('mean')
     .round(3)
     .sort_values("created_at_h") 
     .reset_index()
    )

    stat = stat.rename(columns ={'created_at_h':'time'})
    return stat

In [120]:
path = 'python/tests/unit/fixtures/'
df_sentiments1 = pd.read_csv(glob(path + 'sentiments/*')[0])
df_emotions1 = pd.read_csv(glob(path + 'emotions/*')[0])

In [136]:
stat_sentiments1 = calc_stat_sentiments(df_sentiments1)
stat_sentiments1

,time,mean,count
0,2020-08-25 00:00:00,-0.269,1080
1,2020-08-25 01:00:00,-0.197,938
2,2020-08-25 02:00:00,-0.219,816
3,2020-08-25 03:00:00,-0.161,757
4,2020-08-25 04:00:00,-0.231,828
5,2020-08-25 05:00:00,-0.222,1236
6,2020-08-25 06:00:00,-0.251,1356
7,2020-08-25 07:00:00,-0.228,1379
8,2020-08-25 08:00:00,-0.274,1576
9,2020-08-25 09:00:00,-0.280,1997


In [137]:
stat_sentiments1.to_csv('stat_sentiments1.csv', index=False)


In [123]:
stat_emotions1 = calc_stat_emotions(df_emotions1)
stat_emotions1

,time,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy
0,2020-08-25 00:00:00,0.242,0.208,0.148,0.269,0.195,0.709,0.645,0.096,0.143,0.160
1,2020-08-25 01:00:00,0.271,0.273,0.198,0.322,0.243,0.745,0.656,0.179,0.214,0.214
2,2020-08-25 02:00:00,0.292,0.276,0.206,0.330,0.254,0.741,0.653,0.206,0.218,0.219
3,2020-08-25 03:00:00,0.324,0.273,0.227,0.390,0.273,0.777,0.686,0.214,0.263,0.247
4,2020-08-25 04:00:00,0.326,0.264,0.203,0.350,0.262,0.721,0.705,0.199,0.256,0.225
5,2020-08-25 05:00:00,0.268,0.265,0.208,0.301,0.218,0.616,0.739,0.184,0.191,0.214
6,2020-08-25 06:00:00,0.277,0.281,0.235,0.343,0.265,0.631,0.756,0.176,0.206,0.248
7,2020-08-25 07:00:00,0.303,0.294,0.219,0.336,0.265,0.648,0.753,0.188,0.234,0.221
8,2020-08-25 08:00:00,0.312,0.286,0.216,0.344,0.294,0.648,0.701,0.201,0.244,0.246
9,2020-08-25 09:00:00,0.327,0.306,0.217,0.350,0.296,0.648,0.714,0.195,0.257,0.230


In [138]:
stat_emotions1.to_csv('stat_emotions1.csv', index=False)


In [99]:
import numpy as np 

def calc_stat_words(df_words):
    df_words = df_words.dropna(subset=['token_counter'])

    if len(df_words)==0: return pd.DataFrame(data={'token_counter':[{}],'count':[0]})
    # This may take some time
    stat = (#df_words[['created_at_h', 'token_counter']]
        df_words[['token_counter', 'count']]
     #.groupby('created_at_h') # don't need to group
    # .agg(['sum','count'])
    .agg(['sum'])
     #.sort_values("created_at_h") 
     #.reset_index()
    )

    #stat.columns = stat.columns.droplevel(level = 0)
    #stat.columns = ['time','token_counter','count']

    return stat 

def isCounter(x): 
    return type(x) == type(Counter(('a','b','c')))

def makeVarCounter(df, varname):
    if not isCounter(df[varname][0]):
        df[varname]= df[varname].apply(lambda x: Counter(x))  
    return None
    

def df_words_sample(df_words, batch_num = 100, num_to_sample = 10001):
    ''' 
    prepares data for word cloud as a random sample of data  
    ''' 
    words = deepcopy(df_words)
    makeVarCounter(words, 'token_counter')
    words = words[['token_counter']].dropna()
    
    if len(words)==0: return pd.DataFrame(columns=['token_counter','count'])

    if len(words) < num_to_sample:
        df = words
    else:
        df = words.sample(n=num_to_sample)

    repetitions = np.floor((len(df)/batch_num))
    last = len(df) % batch_num
    idx_beg = 0
    idx_end = batch_num

    agg_words = []

    i = 0
    while i <= repetitions:
        #print('aggregating rows in df_words: ', idx_beg, idx_end)

        a = (df[idx_beg:idx_end]
         #.groupby('created_at_h')
         .agg(['sum', 'count'])
         #.sort_values("created_at_h")
        )
        #a.columns = a.columns.droplevel(level = 0)
        #a = a.reset_index().rename(columns = {'sum':'token_counter'})
        a = a.transpose().rename(columns = {'sum':'token_counter'})
        # Counter(dict())
        # reduce the token_counter to the most common 200 words
        a['token_counter'] = a.token_counter.apply(lambda x: x.most_common(200))
        agg_words.append(a)

        i += 1
        idx_beg += batch_num
        if (i== repetitions - 1): 
            if last==0:
                break
            else:
                idx_end += last
        else:
            idx_end += batch_num

    df_agg_words = pd.concat(agg_words, ignore_index=True)
    df_agg_words['token_counter'] = df_agg_words.token_counter.apply(lambda x: Counter(dict(x)))
    return df_agg_words
    

In [74]:
df_original1 = pd.read_json(glob(path + 'original/*')[0], orient='records', lines=True)
df_retweet1 = pd.read_json(glob(path + 'retweet/*')[0], orient='records', lines=True)
df_words1 = pd.read_json(glob(path + 'words/*')[0], orient='records', lines=True)


In [96]:
from copy import deepcopy
deepcopy?


In [77]:
from collections import Counter
type(df_words.token_counter[6])
type(Counter(df_words.token_counter[6])) == type(Counter(('a','b','c'))) #= df_words.token_counter.apply(lambda x: Counter(x))  

True

In [98]:
def isCounter(x): 
    return type(x) == type(Counter(('a','b','c')))

#print(isCounter(df_words.token_counter[6]))
#isCounter(Counter(df_words.token_counter[6]))
#isCounter(Counter(df_words.token_counter[0]))
# words = deepcopy(df_words)
# makeVarCounter(words, 'token_counter')
# print(isCounter(df_words.token_counter[6]))


In [123]:
def is_xth_column_counter(df, x=0):
    return isCounter(df.iloc[:,x].reset_index(drop=True).get(0))

def sum_df_counter(df, newvarname='token_counter'):
    if len(df.columns) > 1 or not is_xth_column_counter(df,0): 
        raise TypeError(f'"df" must contain only a Counter variable.:df = {df}')
        
    a = df.agg(['sum', 'count'])
    a = a.transpose().rename(columns = {'sum':newvarname})
    return a

def sum_df_counter_most_common(df, newvarname='token_counter', num_most_common=200):
    result = sum_df_counter(df, newvarname)
    result[newvarname] = result[newvarname].apply(lambda x: x.most_common(num_most_common))
    return result


In [125]:
class BatchDfProcessor:
    def __init__(self, df, batch_size):
        self.df = df
        self.batch_size = batch_size
        self.batches = np.floor((len(df)/batch_size))
        self.last_batch_size = len(df) % batch_size
        self.processed_df = pd.DataFrame()
    
    def process(self, func, *args, **kwargs):
        b = 0
        i_beg = 0
        i_end = self.batch_size
        processed = []
        while b < self.batches:
            batch_result = func(self.df[i_beg:i_end], *args, **kwargs)
            print(batch_result)
            processed.append(batch_result)
            b += 1
            i_beg += self.batch_size
            i_end = self.find_next_batch_size(b, i_end)
            if i_end is None: break
        self.processed_df = pd.concat(processed, ignore_index=True)
        print(self.processed_df)
            
    def find_next_batch_size(self, b, i_end):
        if b == self.batches: 
            if self.last_batch_size==0:
                return None
            else:
                i_end += self.last_batch_size
        else:
            i_end += self.batch_size
        return i_end


In [120]:
type(df_words1.token_counter[0])

dict

In [121]:

def make_var_counter(df, varname):
    item1 = df[varname].get(0, True)
    if type(item1)==list:
        df[varname]= df[varname].apply(lambda x: Counter(dict(x)))  
    elif not isCounter(item1): 
        df[varname]= df[varname].apply(lambda x: Counter(x))  
    return None

In [122]:
def df_words_sample_and_batch_sum(df_words, batch_size = 100, num_to_sample = 10000):
    words = deepcopy(df_words)
    make_var_counter(words, 'token_counter')
    words = words[['token_counter']].dropna()
    
    if len(words)==0: return pd.DataFrame(columns=['token_counter','count'])

    if len(words) < num_to_sample:
        df = words
    else:
        df = words.sample(n=num_to_sample)

    batch_df_processor = BatchDfProcessor(df, batch_size)
    batch_df_processor.process(sum_df_counter_most_common, 
                               newvarname='token_counter', 
                               num_most_common=200)
        
    make_var_counter(batch_df_processor.processed_df, 'token_counter')
    return batch_df_processor.processed_df
    
    
def calc_stat_words(df_words, subset_name=''):
    words = df_words.dropna(subset=['token_counter'])

    if len(words)==0: return pd.DataFrame(data={'token_counter':[{}],'count':[0]})
    # This may take some time
    stat_words = words[['token_counter', 'count']].agg(['sum'])
    if subset_name:
        stat_words = add_subset_name(stat_words, subset_name) 
    return stat_words


In [86]:
df_words2 = deepcopy(df_words1)
df_words2.token_counter = df_words2.token_counter.apply(lambda x: Counter(x))
df_words2[:1000][['token_counter']].agg(['sum','count']).transpose()


#calc_stat_words(df_words1)
#calc_stat_words(df_words_sample_and_batch_sum(df_words))


,sum,count
token_counter,"{'trigger': 4, 'warning:': 6, 'straight': 4, '...",1000


In [126]:
tmp = df_words_sample_and_batch_sum(df_words1[:1000])
tmp

                                                   token_counter count
token_counter  [(kenosha,, 18), (back, 11), (#jacobblake, 10)...   100
                                                   token_counter count
token_counter  [(kenosha,, 18), (time, 12), (man, 10), (#keno...   100
                                                   token_counter count
token_counter  [(kenosha,, 14), (police, 11), (time, 9), (bac...   100
                                                   token_counter count
token_counter  [(kenosha,, 16), (black, 13), (man, 12), (vide...   100
                                                   token_counter count
token_counter  [(shoot, 17), (time, 15), (man, 13), (black, 1...   100
                                                   token_counter count
token_counter  [(shoot, 18), (time, 14), (jacob, 11), (man, 1...   100
                                                   token_counter count
token_counter  [(black, 21), (kenosha,, 17), (back, 15), (tim...   100
      

,token_counter,count
0,"{'kenosha,': 18, 'back': 11, '#jacobblake': 10...",100
1,"{'kenosha,': 18, 'time': 12, 'man': 10, '#keno...",100
2,"{'kenosha,': 14, 'police': 11, 'time': 9, 'bac...",100
3,"{'kenosha,': 16, 'black': 13, 'man': 12, 'vide...",100
4,"{'shoot': 17, 'time': 15, 'man': 13, 'black': ...",100
5,"{'shoot': 18, 'time': 14, 'jacob': 11, 'man': ...",100
6,"{'black': 21, 'kenosha,': 17, 'back': 15, 'tim...",100
7,"{'kenosha,': 16, 'time': 11, 'back': 10, 'wi.'...",100
8,"{'kenosha,': 14, 'video': 10, 'street': 9, 'po...",100
9,"{'police': 18, 'black': 14, 'department': 13, ...",100


In [112]:
Counter(dict(tmp.token_counter[0]))

Counter({'kenosha,': 18,
         'back': 11,
         '#jacobblake': 10,
         'video': 9,
         'time': 9,
         'wisc.': 8,
         'riot.': 8,
         'street': 7,
         'say': 7,
         'wi.': 6,
         'shot': 6,
         'massive': 6,
         'fire': 6,
         'race': 6,
         '29-year-old': 6,
         'jacob': 6,
         'shoot': 6,
         'share': 6,
         'black': 5,
         'man': 5,
         '#kenosha': 5,
         'happend': 5,
         '💔': 5,
         '#blacklivesmatter.': 5,
         'keep': 4,
         'happening.': 4,
         'video.': 4,
         'one': 4,
         'watch': 4,
         'that.': 4,
         'beautiful': 4,
         'photo': 4,
         'blake,': 4,
         'seven': 4,
         'police': 4,
         'broad': 4,
         'daylight': 4,
         'wisconsin.': 4,
         'name.': 4,
         'bastard': 4,
         'murderer': 4,
         'piece': 4,
         'shit': 4,
         'blm': 3,
         'several': 3,
         '

In [390]:
def get_top_tweets(ori_data, cum_rt_data, subset_name=''):
    #print('IN get_top_tweets():')
    df_null = pd.DataFrame(
      data = {'subset':[subset_name], 'RT_id':[''],
      'user_name':[''], 'followers_count':[''], 'text':[''], 't_co':[''],
       'tags':[''], 'retweet_timespan':[''], 'retweet_total':['']}
      )
    if len(ori_data)==0: return df_null 
    len_top = max(15, len(ori_data))

    top15_TW = ori_data.RT_id.value_counts()[:(len_top + 1)]
    top15_TW = top15_TW[top15_TW.index!=''][:len_top]
    top15_TW.index = top15_TW.index.astype(str)
    # print(top15_TW)

    top_RT = cum_rt_data[['RT_id','user_name','followers_count','text','t_co','tags']]
    top_RT.RT_id = top_RT.RT_id.astype(str)

    idx1 = [id in top15_TW.keys() for id in top_RT.RT_id]
    if sum(idx1)==0: return df_null
    top_RT = top_RT[idx1].drop_duplicates(subset=['RT_id'])

    top_TW_count = ori_data[['RT_id','RT_retweet_count']][ori_data.RT_id != '']
    idx2 = [id in top15_TW.keys() for id in top_TW_count.RT_id]

    top_TW_count = (top_TW_count.loc[idx2]
                    .groupby('RT_id')
                    .agg(['count', 'max'])
                    .reset_index()
                   )
    top_TW_count.columns = top_TW_count.columns.droplevel(level=0)
    top_TW_count = top_TW_count.rename(columns={'':'RT_id','count':'retweet_timespan','max':'retweet_total'})

    top_tweets = (top_RT.set_index('RT_id')
     .join(top_TW_count.set_index('RT_id'))
     .sort_values(by=['retweet_timespan'], ascending=False)
     .reset_index()
    )
    if (subset_name != ''):
        cols = top_tweets.columns
        top_tweets['subset'] = subset_name 
        top_tweets = top_tweets[['subset', *cols]]
    return top_tweets


#tmp = get_top_tweets(cum_data.cum_ori.now_1h, cum_data.cum_rt, subset_name='now_1h')
#tmp


def get_top_users(ori_data, cum_rt, subset_name=''):
    ori_data = ori_data[['RT_id']][ori_data.RT_id !=''] 

    merged = (ori_data.set_index('RT_id')  
           .join(cum_rt[['RT_id','user_id','user_name','user_description',
                            'followers_count','following_count']].set_index('RT_id'),
                rsuffix='_RT')
          .dropna(subset=['user_id'])
          )
    merged.user_id = merged.user_id.astype(int).astype(str)
    top15_users = merged.user_id.value_counts()[:15]

    idx1 = [id in [*top15_users.keys()] for id in merged.user_id]  
    
    if (len(idx1)==0): 
      df_null = pd.DataFrame(columns = 
        ['subset', 'user_id', 'index', 'RT_id', 'user_name', 'user_description',
       'followers_count', 'following_count', 'retweeted'])
      df_null['subset'] = subset_name 
      return df_null

    merged2 = (merged[idx1].reset_index()
     .sort_values(by=['followers_count','RT_id'], ascending=False)
     .drop_duplicates(subset = 'user_id')
     .reset_index()
    )
    
    top_users = (merged2.set_index('user_id')
          .join(top15_users)
          .rename(columns = {'user_id':'retweeted'})
          .reset_index()
          .sort_values(by = 'retweeted', ascending = False)
         )
    
    if (subset_name != ''):
        cols = top_users.columns
        top_users['subset'] = subset_name 
        top_users = top_users[['subset', *cols]]
        
    return top_users



In [412]:

def remove_null_on_str_var(df, varname, null=''):
    return df[df[varname]!=null]

def remove_null_on_num_var(df, varname, null=''):
    return df[df[varname].notnull()]

def get_top_counts_cases(df, varname, n_top = 15):
    return df[varname].value_counts()[:n_top]


def isStr(x): return type(x)==str
def isNum(x): 
    try:
        return type(round(x))==int
    except:
        return False

def tolist(x):
    if isStr(x) or isNum(x): return [x]
    else: return list(x)
    
def convert_vars_to_str(df, vars):
    for var in tolist(vars):
        df[var] = df[var].astype(str)
    return None

def convert_index_to_str(df):
    df.index = df.index.astype(str)
    return None

def drop_duplicates_index(df, keep='first'):
    tmp = df.reset_index()
    index_var = tmp.columns[0]
    tmp.drop_duplicates(subset = index_var, keep=keep, inplace=True)
    return tmp.set_index(index_var)

# tmp = get_top_counts_cases_str(df_original1, 'RT_id')
# convert_index_to_str(tmp)
# tmp.index

[1, 2, 3]

In [448]:
from cached_property import cached_property

class TopTweetsCalculator:
    def __init__(self, ori_data, cum_rt_data, num_top_tweets=15):
        convert_vars_to_str(ori_data, 'RT_id')
        convert_vars_to_str(cum_rt_data, 'RT_id')
        self.ori_data = remove_null_on_str_var(ori_data, 'RT_id')
        self.cum_rt_data = cum_rt_data
        self.num_top_tweets = min(num_top_tweets, len(ori_data))

    @property
    def df_null(self):
        df_null = pd.DataFrame(
                    columns=['RT_id','user_name', 'followers_count', 'text', 't_co',
                       'tags', 'retweet_timespan', 'retweet_total'])
        return df_null
    
    @cached_property
    def top_RT_ids(self):
        top_RT_ids = get_top_counts_cases(self.ori_data, 'RT_id', self.num_top_tweets)
        return top_RT_ids
    
    @cached_property
    def top_RT_data(self):
        top_RT_ids_no_missing = [id for id in self.top_RT_ids.index if id in list(self.cum_rt_data.RT_id)]
        top_RT_data = (self.cum_rt_data[['RT_id','user_name','followers_count','text','t_co','tags']]
                       .set_index('RT_id')
                       .loc[top_RT_ids_no_missing]
                       )
        if len(top_RT_data)==0: return None
        top_RT_data = drop_duplicates_index(top_RT_data, keep='last') 
        return top_RT_data
    
    @cached_property
    def top_RT_stats(self):
        top_RT_stats = (self.ori_data[['RT_id','RT_retweet_count']]
                        .set_index('RT_id')
                        .loc[self.top_RT_ids.index]
                        .reset_index()
                        .groupby('RT_id')
                        .agg(['count', 'max'])
                        .reset_index()
                       )
        top_RT_stats.columns = top_RT_stats.columns.droplevel(level=0)
        top_RT_stats = (top_RT_stats
                        .rename(columns={'':'RT_id', 
                                         'count':'retweet_timespan',
                                         'max':'retweet_total'})
                        .set_index('RT_id')
                       )
        return top_RT_stats
    
    @cached_property
    def top_tweets(self):
        if len(self.ori_data)==0 or self.top_RT_data is None: return self.df_null
        top_tweets = (
            self.top_RT_data
            .join(self.top_RT_stats)
            .sort_values(by=['retweet_timespan'], ascending=False)
            .reset_index()
            )
        return top_tweets
    

    
    
def bring_a_column_first(df, varname):
    cols = [col for col in df.columns if col != varname]
    return df[[varname, *cols]]

def add_subset_name(df, subset_name):
    df['subset_name'] = subset_name
    return bring_a_column_first(df, 'subset_name')


def calc_top_tweets(ori_data, cum_rt_data, subset_name='', num_top_tweets=15): 
    top_tweets_calc = TopTweetsCalculator(ori_data, cum_rt_data, num_top_tweets)
    top_tweets = top_tweets_calc.top_tweets
    if subset_name:
        top_tweets = add_subset_name(top_tweets, subset_name)
    return top_tweets
    

In [15]:
tmp = calc_top_tweets(df_original1, df_retweet1, subset_name='today')
tmp

,subset_name,RT_id,user_name,followers_count,text,t_co,tags,retweet_timespan,retweet_total
0,today,1297830004781268992,aishatyler,504510,He was trying to break up a fight.\n\nHe wasn’...,https://t.co/YPny1mXUOP,[#blm],5246,58794
1,today,1297979490933645312,KingJames,47412066,And y’all wonder why we say what we say about ...,https://t.co/TGtIXbq4Sv,[],4794,52877
2,today,1297927031230009344,spidadmitchell,771721,F THE GAMES AND PLAYOFFS!!! THIS IS SICK AND I...,https://t.co/q780ZvgwB9,[],4327,46486
3,today,1297875830111178752,iamshleigh,175,SAY HIS NAME. Jacob Blake was shot 7 times in ...,https://t.co/poqkSRYOih,[#BlackLivesMatter],651,6781


In [16]:
tmp.followers_count

0      504510
1    47412066
2      771721
3         175
Name: followers_count, dtype: int64

In [177]:
class TopUsersCalculator:
    def __init__(self, ori_data, cum_rt_data, num_top_users=15):
        convert_vars_to_str(ori_data, 'RT_id')
        convert_vars_to_str(cum_rt_data, ['RT_id','user_id'])
        self.ori_data = remove_null_on_str_var(ori_data, 'RT_id')
        self.cum_rt_data = cum_rt_data
        self.num_top_users = min(num_top_users, len(ori_data))
    
    @cached_property
    def top_RT_ids(self):
        RT_id_to_user_id_coverage_factor = 3
        top_RT_ids = get_top_counts_cases(self.ori_data, 'RT_id', 
                                          self.num_top_users * RT_id_to_user_id_coverage_factor)
        return top_RT_ids
    
    @cached_property
    def userdata_of_top_RT(self):
        userdata_of_top_RT = (
            self.ori_data[['RT_id']].set_index('RT_id').loc[self.top_RT_ids.index]  
            .join(self.cum_rt_data[['RT_id','user_id','user_name','user_description',
                                    'followers_count','following_count']].set_index('RT_id'),
                  how='inner')
            .reset_index()
            .dropna(subset=['user_id'])
            )
        return userdata_of_top_RT
    
    @cached_property
    def top_users_ids(self):
        top_user_ids = get_top_counts_cases(self.userdata_of_top_RT, 'user_id', self.num_top_users) 
        return top_user_ids

    @cached_property
    def top_users_data(self):
        top_users_data = (
            self.userdata_of_top_RT.set_index('user_id').loc[self.top_users_ids.index]
            .reset_index()
            .drop_duplicates(subset = 'user_id', keep='last')
        )
        if len(top_users_data)==0: return None
        return top_users_data
    
    @cached_property
    def top_users(self):
        if len(self.ori_data)==0 or self.top_users_data is None: return self.df_null
        top_users = (self.top_users_data.set_index('user_id')
          .join(self.top_users_ids)
          .rename(columns = {'user_id':'retweeted'})
          .reset_index()
          .sort_values(by = ['retweeted','followers_count'], ascending = False)
         )
        return top_users
    
    @property
    def df_null(self):
        df_null = pd.DataFrame(
            columns = ['user_id', 'RT_id', 'user_name', 'user_description',
                       'followers_count', 'following_count', 'retweeted'])
        return df_null
    
    

def calc_top_users(ori_data, cum_rt_data, subset_name='', num_top_users=15):
    top_users_calc = TopUsersCalculator(ori_data, cum_rt_data, num_top_users)
    top_users = top_users_calc.top_users
    if subset_name:
        top_users = add_subset_name(top_users, subset_name)  
    return top_users


In [141]:
tmp2 = calc_top_users(df_original1, df_retweet1, subset_name='today')
tmp2

,subset_name,user_id,RT_id,user_name,user_description,followers_count,following_count,retweeted
0,today,23083404,1297979490933645312,KingJames,EST. AKRON - ST.V/M Class of '03 http://LeBro...,47412066,190,9588
1,today,18125335,1297830004781268992,aishatyler,director. actor. writer. cocktail enthusiast. ...,504510,783,5246
2,today,488026266,1297927031230009344,spidadmitchell,Be Humble🙏🏾 Former Louisville Guard #L1C4... U...,771721,1014,4327
3,today,2835451658,1297912931699404800,MrAndyNgo,"Editor-at-large - @TPostMillennial. ""Unmasked""...",708512,739,1124
4,today,1045110787,1298050385614077952,MaryLTrump,"#blacklivesmatter, 🏳️‍🌈,she/her/hers",578330,904,832
5,today,703645297713553408,1297875830111178752,iamshleigh,Currently @PartnersGlobal. Former @USUN and @U...,175,687,651
6,today,1138076821533155328,1297865500098064384,JamaalBowmanNY,Democratic nominee for #NY16. Former middle sc...,168690,1548,607
7,today,90480218,1297875519308890112,RichardGrenell,I’m stronger after cancer. my dog runs my life...,538877,2650,587
8,today,7391432,1297724068611993600,elizaorlins,Manhattan DA Candidate. NYC public defender fo...,83196,6395,564
9,today,350822109,1298101789716295680,ElijahSchaffer,EXPOSE DARKNESS 🇺🇸 | @blazetv Host & Reporter ...,177501,461,560


In [ ]:
class placeholder(): pass

def time_subsets_all(df, now=None):
        now, now_1h, today, yesterday, seven_d_ago = get_cum_times(now)
        
        #df.created_at_h = pd.to_datetime(df.created_at_h)
        df['created_at_d'] = df.created_at_h.dt.floor('d')
        df['hour'] = df.created_at_h.apply(lambda x: int(str(x)[11:13]))

        subsets = placeholder()
        for h in range(24):
          setattr(subsets,'hour_' + str(h), df[df.hour==h]) 

        subsets.today = df[df.created_at_d==today]
        subsets.yesterday = df[df.created_at_d==yesterday]
        subsets.seven_days = df[(df.created_at_d <= today) & (df.created_at_d >=seven_d_ago)]
        return(subsets)

class cumulative_data2():
    def __init__(self, cum_ori, cum_rt, cum_words, now=None):
        self.cum_ori = time_subsets_all(cum_ori, now=now)
        self.cum_rt = cum_rt
        self.cum_words = time_subsets_all(cum_words, now=now)
        self.now = now
        sefl.subset_names = ['today', 'yesterday', 'seven_days'] + ['hour_' + str(h) for h in range(24)]

    def add_words_subsets(self):
        subs = self.cum_words
        stat_words = []
        for subset in selfsubset_names:
            stat_words.append(calc_stat_words(df_words_sample(getattr(subs, subset)),
                                              subset_name=subset))
        
        return pd.pd.concat(processed, ignore_index=True)
        
        stat_words = (calc_stat_words(df_words_sample(subs.today)).
                      append(calc_stat_words(df_words_sample(subs.yesterday))).
                      append(calc_stat_words(df_words_sample(subs.seven_days)))
                     )
        
        for h in range(24):
          stat_words = stat_words.append(
            calc_stat_words(df_words_sample(getattr(subs,'hour_' + str(h))))
            )

        stat_words.index = ['today', 'yesterday', 'seven_days'] + ['hour_' + str(h) for h in range(24)]
        self.stat_words = stat_words.reset_index().rename(columns = {'index': 'subset'})
        
    def add_tweet_subsets(self):
        subs = self.cum_ori
        cum_rt = self.cum_rt

        top_tweets = (get_top_tweets(subs.today, cum_rt, subset_name='today')
                  .append(get_top_tweets(subs.yesterday, cum_rt, subset_name='yesterday'))
                  .append(get_top_tweets(subs.seven_days, cum_rt, subset_name='seven_days'))
                 )

        for h in range(24):
          hour_h = 'hour_' + str(h)
          top_tweets = top_tweets.append(
            get_top_tweets(getattr(subs, hour_h), cum_rt, subset_name=hour_h)
          )
        self.top_tweets = top_tweets
        
    def add_user_subsets(self):
        subs = self.cum_ori
        cum_rt = self.cum_rt

        top_users = (get_top_users(subs.today, cum_rt, subset_name='today')
                      .append(get_top_users(subs.yesterday, cum_rt, subset_name='yesterday'))
                      .append(get_top_users(subs.seven_days, cum_rt, subset_name='seven_days'))
                     )
        for h in range(24):
          hour_h = 'hour_' + str(h)
          top_users = top_users.append(
            get_top_users(getattr(subs, hour_h), cum_rt, subset_name=hour_h)
            )
        self.top_users = top_users
        



    


In [538]:
type(df_original1.created_at_h[0]) == type(pd.to_datetime('2020-09-17'))
type(pd.to_datetime(df_original1.created_at_h[0]))# == type(pd.to_datetime('2020-09-17'))

def isPandasTimestamp(x):
    return type(x)==type(pd.to_datetime('2020-09-17'))

def make_var_pandas_timestamp(df, varname):
    if not isPandasTimestamp(df[varname][0]):
        df[varname]= pd.to_datetime(df[varname])
    return None

def extract_hour_from_pandas_timestamp(df, varname):
    make_var_pandas_timestamp(df, varname)
    return df[varname].apply(lambda x: x.hour)
    

In [539]:
tmp = pd.to_datetime(df_original1.created_at_h[0])
tmp.day

tmp1 = extract_hour_from_pandas_timestamp(df_original1, 'created_at_h')
tmp1

0         0
1         0
2         0
3         0
4         0
         ..
77051    23
77052    23
77053    23
77054    23
77055    23
Name: created_at_h, Length: 77056, dtype: int64

In [15]:
#pd.Timestamp(time_now(tzname='America/Denver'))
get_subset_times()

def time_now_pandas(tzname='America/Denver'):
    now = time_now(tzname)
    return pd.to_datetime(now.strftime('%Y-%m-%d %H:%M:%S')) 
    
time_now_pandas()

Timestamp('2020-09-17 14:20:15')

In [1]:
import sys
sys.path.append('python/')

from copy import deepcopy
import pandas as pd


from utilities import time_now_pandas, extract_hour_from_pandas_timestamp, \
     make_var_pandas_timestamp, PlaceHolder
from tweetRetweetStats import * 


def get_subset_times(now=None):
    if now is None:
        now = time_now_pandas(tzname='America/Denver')
    
    now_1h = now + pd.DateOffset(hours=-1)
    today = now.floor("d")
    yesterday = today + pd.DateOffset(days=-1)
    seven_d_ago = today + pd.DateOffset(days=-7)    
    return now, now_1h, today, yesterday, seven_d_ago

def create_time_subsets_of_df(df, timevar='created_at_h', now=None):
    now, now_1h, today, yesterday, seven_d_ago = get_subset_times(now)
    
    df = deepcopy(df)
    make_var_pandas_timestamp(df, timevar)
    if hasattr(df_original1, '_timevar_d') or hasattr(df_original1, '_hour'):
        raise AttributeError('Variable name already exists: avoid names "_timevar_d" and "_hour".')
    df['_timevar_d'] = df[timevar].dt.floor('d')
    df['_hour'] = extract_hour_from_pandas_timestamp(df, timevar)

    subsets = PlaceHolder()
    idx_today = df._timevar_d == today
    for h in range(24):
        setattr(subsets,'hour_' + str(h), df[(df._hour==h) & (idx_today)]) 

    subsets.today = df[df._timevar_d==today]
    subsets.yesterday = df[df._timevar_d==yesterday]
    subsets.seven_days = df[(df._timevar_d <= today) & (df._timevar_d >=seven_d_ago)]
    return(subsets)

    
class TweetRetweetData:
    def __init__(self, cum_ori, cum_rt, cum_words, now=None):
        self.cum_ori = create_time_subsets_of_df(cum_ori, timevar='created_at_h', now=now)
        self.cum_rt = cum_rt
        self.cum_words = create_time_subsets_of_df(cum_words, timevar='created_at_h', now=now)
        self.now = now
        self.subset_names = ['today', 'yesterday', 'seven_days'] + ['hour_' + str(h) for h in range(24)]

    @cached_property
    def stat_words(self):
        subs = self.cum_words
        stat_words = []
        for subset in self.subset_names:
            stat_words.append(calc_stat_words(
                df_words_sample_and_batch_sum(getattr(subs, subset)),
                subset_name=subset))
        return pd.concat(stat_words, ignore_index=True)

    @cached_property
    def top_tweets(self):
        subs = self.cum_ori
        cum_rt = self.cum_rt
        top_tweets = []
        for subset in self.subset_names:
            top_tweets.append(calc_top_tweets(getattr(subs, subset), cum_rt, subset_name=subset))
        return pd.concat(top_tweets, ignore_index=True)

    @cached_property
    def top_users(self):
        subs = self.cum_ori
        cum_rt = self.cum_rt
        top_users = []
        for subset in self.subset_names:
            top_users.append(calc_top_users(getattr(subs, subset), cum_rt, subset_name=subset))
        return pd.concat(top_users, ignore_index=True)


In [2]:
from glob import glob
path = 'python/tests/unit/fixtures/'

df_original1 = pd.read_json(glob(path + 'original/*')[0], orient='records', lines=True)
df_retweet1 = pd.read_json(glob(path + 'retweet/*')[0], orient='records', lines=True)
df_words1 = pd.read_json(glob(path + 'words/*')[0], orient='records', lines=True)


In [3]:
base_timestamp = pd.Timestamp('2020-08-24-22')
tweet_retweet_data = TweetRetweetData(df_original1, df_retweet1, df_words1, now=base_timestamp)


In [4]:
tweet_retweet_data.stat_words

,subset_name,token_counter,count
0,today,"{'say': 1825, 'shoot': 1194, 'jacob': 1248, 'd...",10000
1,yesterday,{},0
2,seven_days,"{'man': 1549, 'demand': 1353, 'black': 1723, '...",10000
3,hour_0,"{'kenosha,': 446, 'back': 305, '#jacobblake': ...",3700
4,hour_1,"{'time': 261, 'back': 250, 'shot': 150, 'video...",3100
5,hour_2,"{'time': 194, 'black': 191, 'back': 199, 'car'...",2600
6,hour_3,"{'kenosha,': 339, 'black': 231, 'shoot': 214, ...",2800
7,hour_4,"{'kenosha,': 336, 'back': 212, 'time': 249, 'p...",3000
8,hour_5,"{'kenosha,': 533, 'black': 362, 'shoot': 428, ...",4200
9,hour_6,"{'kenosha,': 535, 'black': 694, 'wi.': 268, 't...",7100


In [6]:
top_tweets = tweet_retweet_data.top_tweets
top_tweets

,subset_name,RT_id,user_name,followers_count,text,t_co,tags,retweet_timespan,retweet_total
0,today,1297830004781268992,aishatyler,504510,He was trying to break up a fight.\n\nHe wasn’...,https://t.co/YPny1mXUOP,[#blm],5246,58794
1,today,1297979490933645312,KingJames,47412066,And y’all wonder why we say what we say about ...,https://t.co/TGtIXbq4Sv,[],4794,52877
2,today,1297927031230009344,spidadmitchell,771721,F THE GAMES AND PLAYOFFS!!! THIS IS SICK AND I...,https://t.co/q780ZvgwB9,[],4327,46486
3,today,1297875830111178752,iamshleigh,175,SAY HIS NAME. Jacob Blake was shot 7 times in ...,https://t.co/poqkSRYOih,[#BlackLivesMatter],651,6781
4,seven_days,1297830004781268992,aishatyler,504510,He was trying to break up a fight.\n\nHe wasn’...,https://t.co/YPny1mXUOP,[#blm],5246,58794
...,...,...,...,...,...,...,...,...,...
146,hour_23,1298050385614077952,MaryLTrump,578330,#BlackLivesMatter It must be said because it i...,https://t.co/h2eYfkNV7R,"[#BlackLivesMatter, #SayTheirNames, #GeorgeFlo...",20,4545
147,hour_23,1298106966506692608,theblaze,738076,BREAKING: #BLM rioters light multiple building...,https://t.co/GGgnOOchHa,[#BLM],19,702
148,hour_23,1298084607527534592,MrAndyNgo,609494,"April Lynn Epperson, 36, was arrested at the v...",https://t.co/0IYJ8iYH91,"[#antifa, #PortlandRiots, #PortlandMugshots, #...",17,747
149,hour_23,1298108413734293504,benshapiro,3010834,Those cars were systemically racist https://t....,https://t.co/D3R9C0zhEN,[],16,667


In [7]:
tweet_retweet_data.top_users

,subset_name,user_id,RT_id,user_name,user_description,followers_count,following_count,retweeted
0,today,23083404,1297979490933645312,KingJames,EST. AKRON - ST.V/M Class of '03 http://LeBro...,47412066,190,9588
1,today,18125335,1297830004781268992,aishatyler,director. actor. writer. cocktail enthusiast. ...,504510,783,5246
2,today,488026266,1297927031230009344,spidadmitchell,Be Humble🙏🏾 Former Louisville Guard #L1C4... U...,771721,1014,4327
3,today,2835451658,1297912931699404800,MrAndyNgo,"Editor-at-large - @TPostMillennial. ""Unmasked""...",708512,739,1124
4,today,1045110787,1298050385614077952,MaryLTrump,"#blacklivesmatter, 🏳️‍🌈,she/her/hers",578330,904,832
...,...,...,...,...,...,...,...,...
374,hour_23,592730371,1298131125487927296,JackPosobiec,Veteran. Catholic. Father.,968566,988,39
375,hour_23,10774652,1298106966506692608,theblaze,News and entertainment for people who love Ame...,738076,694,19
376,hour_23,88426758,1297992128602222592,SJPFISH,"RT’d 8 X’s by @realDonaldTrump,@dBongino, Foll...",79864,59665,9
377,hour_23,1269386879482421248,1298109615851425792,Rosemvmt,POC organizers + socialists committed to the g...,11461,537,6


In [8]:
tweet_retweet_data.stat_words.to_json('stat_words1.json', orient='records', lines=True)
tweet_retweet_data.top_tweets.to_json('top_tweets1.json', orient='records', lines=True)
tweet_retweet_data.top_users.to_json('top_users1.json', orient='records', lines=True)


In [5]:
tweet_retweet_data.stat_words.to_json('stat_words1.json', orient='records', lines=True)


In [150]:
tweet_retweet_data.top_users['index']

[   subset_name              user_id                RT_id       user_name  \
0        today             23083404  1297979490933645312       KingJames   
1        today             18125335  1297830004781268992      aishatyler   
2        today            488026266  1297927031230009344  spidadmitchell   
3        today           2835451658  1297912931699404800       MrAndyNgo   
4        today           1045110787  1298050385614077952      MaryLTrump   
5        today   703645297713553408  1297875830111178752      iamshleigh   
6        today  1138076821533155328  1297865500098064384  JamaalBowmanNY   
7        today             90480218  1297875519308890112  RichardGrenell   
8        today              7391432  1297724068611993600     elizaorlins   
9        today            350822109  1298101789716295680  ElijahSchaffer   
10       today            164060400  1297718194551975936     kirkacevedo   
11       today             52338305  1297726733303635968   BrandonStraka   
12       to

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
374    NaN
375    NaN
376    NaN
377    NaN
378    NaN
Name: index, Length: 379, dtype: object

In [152]:
tweet_retweet_data.top_users['index'].value_counts()

Series([], Name: index, dtype: int64)

In [169]:
df1 = []
df2 = pd.DataFrame(columns=['a','b'])
df3 = pd.DataFrame.from_dict({'a':[1],'b':[2]})


df1.append(df2)
df1.append(df3)
df1.append(df2)
df1.append(df3)

In [168]:
pd.concat(df1,ignore_index=True)

,a,b
0,1,2
1,1,2


In [176]:
df1

[Empty DataFrame
 Columns: [a, b]
 Index: [],
    a  b
 0  1  2,
 Empty DataFrame
 Columns: [a, b]
 Index: [],
    a  b
 0  1  2]

In [5]:
from pandas.testing import assert_frame_equal

In [14]:
#tweet_retweet_data.top_tweets.followers_count
#df_retweet1.followers_count

0           41047
1           27362
2          347101
3            1436
4          190921
           ...   
24077       20969
24078       23215
24079      295210
24080       11528
24081    55784668
Name: followers_count, Length: 24082, dtype: int64

In [31]:
tweet_retweet_data.top_users.followers_count.astype(int)

0      47412066
1        504510
2        771721
3        708512
4        578330
         ...   
374      968566
375      738076
376       79864
377       11461
378       26427
Name: followers_count, Length: 379, dtype: int64

In [11]:
tweet_retweet_data.stat_words.token_counter


0     {'say': 1825, 'shoot': 1194, 'jacob': 1248, 'd...
1                                                    {}
2     {'man': 1549, 'demand': 1353, 'black': 1723, '...
3     {'kenosha,': 446, 'back': 305, '#jacobblake': ...
4     {'time': 261, 'back': 250, 'shot': 150, 'video...
5     {'time': 194, 'black': 191, 'back': 199, 'car'...
6     {'kenosha,': 339, 'black': 231, 'shoot': 214, ...
7     {'kenosha,': 336, 'back': 212, 'time': 249, 'p...
8     {'kenosha,': 533, 'black': 362, 'shoot': 428, ...
9     {'kenosha,': 535, 'black': 694, 'wi.': 268, 't...
10    {'shoot': 1561, 'police': 1381, 'jacob': 1829,...
11    {'jacob': 2104, 'time': 1265, 'car': 1615, 'po...
12    {'jacob': 2244, 'shoot': 2173, 'police': 2236,...
13    {'police': 1810, 'break': 2133, 'get': 2102, '...
14    {'car': 2125, 'dont': 2030, 'break': 2127, 'ge...
15    {'demand': 2353, 'justice!': 2314, 'dont': 254...
16    {'car': 2213, 'get': 2253, 'break': 2238, 'dem...
17    {'say': 3568, 'black': 3410, 'man': 3355, 

In [6]:
stat_words1 = pd.read_json('stat_words1.json', orient='records', lines=True)

In [7]:
from pandas.testing import assert_frame_equal

assert_frame_equal(tweet_retweet_data.stat_words, stat_words1)

AssertionError: DataFrame.iloc[:, 1] (column name="token_counter") are different

DataFrame.iloc[:, 1] (column name="token_counter") values are different (33.33333 %)
[left]:  [{'say': 1825, 'shoot': 1194, 'jacob': 1248, 'dont': 966, 'police': 1228, 'get': 1599, 'break': 1614, 'demand': 1369, 'car': 1618, 'name.': 785, '#blm': 1026, 'justice!': 1312, 'wtf': 902, 'feel': 900, 'time': 1299, 'seven': 796, 'front': 877, 'black': 1789, 'handle': 674, 'go': 787, 'back.': 801, 'children.': 772, 'man': 1583, 'back': 1178, 'try': 706, 'fight.': 713, 'wasn’t': 636, 'armed.': 631, 'relieve': 631, 'arrive': 645, 'fight,': 631, 'drive': 645, 'child': 776, 'safely': 644, 'home.': 645, 'times.': 672, 'blake.': 733, 'f': 479, 'games': 476, 'playoffs!!!': 476, 'sick': 485, 'real': 511, 'problem': 485, 'crazy': 476, 'words': 477, 'man!': 477, 'safe!!!!': 476, 'video': 334, 'kenosha,': 459, 'blm': 341, 'shot': 1076, '7': 332, 'tell': 507, 'fight': 937, 'street': 302, 'blake': 313, 'need': 225, 'two': 874

In [28]:
all(list(tweet_retweet_data.stat_words.columns == stat_words1.columns) + \
 [len(tweet_retweet_data.stat_words) == len(stat_words1)])

True

In [29]:
import pytest

In [9]:
top_tweets1 = pd.read_json('top_tweets1.json', orient='records', lines=True)
top_users1 = pd.read_json('top_users1.json', orient='records', lines=True)

In [17]:
top_tweets1['RT_id'] = top_tweets1['RT_id'].astype(str)
tweet_retweet_data.top_tweets['followers_count'] = tweet_retweet_data.top_tweets['followers_count'].astype(int)
tweet_retweet_data.top_tweets['retweet_timespan'] = tweet_retweet_data.top_tweets['retweet_timespan'].astype(int)
tweet_retweet_data.top_tweets['retweet_total'] = tweet_retweet_data.top_tweets['retweet_total'].astype(int)


In [18]:
assert_frame_equal(tweet_retweet_data.top_tweets, top_tweets1)

In [24]:
top_users1['user_id'] = top_users1['user_id'].astype(str)
top_users1['RT_id'] = top_users1['RT_id'].astype(str)
tweet_retweet_data.top_users['followers_count'] = tweet_retweet_data.top_users['followers_count'].astype(int)
tweet_retweet_data.top_users['following_count'] = tweet_retweet_data.top_users['following_count'].astype(int)
tweet_retweet_data.top_users['retweeted'] = tweet_retweet_data.top_users['retweeted'].astype(int)

In [25]:
assert_frame_equal(tweet_retweet_data.top_users, top_users1)

In [30]:
class placeholder: pass
tmp = placeholder()

In [37]:
tmp = dict()

In [38]:
tmp['a'] = 'a'
tmp['b'] = 'b'

In [41]:
[print(i,v) for i,v in tmp.items()]


a a
b b


[None, None]

In [68]:
folders = ('original','retweet','sentiments','emotions','words')
folders

('original', 'retweet', 'sentiments', 'emotions', 'words')

In [56]:
folders + ('retweet',)

('original', 'retweet', 'sentiments', 'emotions', 'words', 'retweet')

In [73]:
for folder in folders:
    if folder!='retweet': print(folder) 

original
sentiments
emotions
words


In [67]:
folders = ['original','retweet','sentiments','emotions','words']


In [74]:
?setattr

In [75]:
tmp = dict()

In [76]:
tmp['a'] = 'a'

In [77]:
setattr(tmp,'b', 'b')

AttributeError: 'dict' object has no attribute 'b'

In [89]:
def isStr(x): return type(x)==str

def isNum(x): 
    try:
        return type(round(x))==int
    except:
        return False

def tolist(x):
    if isStr(x) or isNum(x): return [x]
    else: return list(x)
    
def remove_list_items_ending_with(items, str_or_iterable):
    iterlist = tolist(str_or_iterable)
    return [item for item in items if not any([item.endswith(s) for s in iterlist])]
    
    

In [91]:
list1 = ['app_data/data_cumulative/city_date/all_v1/original/.DS_Store', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-05-27.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-05-30.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-03.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-06.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-10.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-13.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-17.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-20.json', 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-24.json']
remove_listitems_ends_with(list1, '.DS_Store')


['app_data/data_cumulative/city_date/all_v1/original/records_2020-05-27.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-05-30.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-03.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-06.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-10.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-13.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-17.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-20.json',
 'app_data/data_cumulative/city_date/all_v1/original/records_2020-06-24.json']

In [81]:
'app_data/data_cumulative/city_date/all_v1/original/.DS_Store'.endswith('.DS_Store','a']

TypeError: endswith first arg must be str or a tuple of str, not list

In [105]:
import csv

tmp = csv.reader('sentiments1.csv')


In [103]:
from io import BytesIO


NameError: name 'Byte' is not defined

In [110]:
import json
with open('stat_sentiments1.csv') as f:
    data = json.load(f)
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [113]:
with open('stat_sentiments1.csv') as f:
    data = [i for i in csv.reader(f)]

In [119]:
pd.DataFrame(pd.concat(data, ignore_index=True))

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [126]:
if len(list([])): print('ya')

In [124]:
dict1 = {'a': [1,2]}
if dict1.get('a'): print('a')

a
